In [1]:
import tensorflow as tf

# 对比对logits进行不同次softmax：
正常情况下，非概率式的原始logits，需要经过softmax处理，再用来训练和预测
经过第二次的softmax后，分布概率会有变化，虽然大小关系不变，不影响argmax直接做预测，但是训练的话，数值意义就不同了，明显二次softmax之后，数值的相对差距平缓了，训练难度增加！！！


In [14]:
#虚拟两个数据。one-hot式的label
#第一个label是下标2,第一个数据最大的下标是2,对应，所以loss期望小。
#第二个label是下标1,而数据最大下标是2,不对应，期望loss大
labels = [[0,0,1],[0,1,0]]
logits = [[2,  0.5,6],
          [0.1,0,  3]]
logits_scaled = tf.nn.softmax(logits)
logits_scaled2 = tf.nn.softmax(logits_scaled)
#交叉熵支持直接使用logits，现在用logits和softmax处理过的logits对比，还有手动交叉熵作为参照
loss1 = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
loss2 = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits_scaled)
loss3 = -tf.reduce_sum(labels*tf.log(logits_scaled),1)
with tf.Session() as sess:
    print ("original=",logits)  
    print ("scaled=",sess.run(logits_scaled))    
    print ("scaled2=",sess.run(logits_scaled2)) 
    
    print ("loss1=",sess.run(loss1),"\n")#正确的方式
    #如果将softmax变换完的值放进去会，重复softmax，缓和了不同概率之间的差距
    #也让本该大的loss变小，本该小的loss变大，严重影响训练。
    print ("loss2=",sess.run(loss2),"\n")
    print ("loss3=",sess.run(loss3))
    print('total_loss1:',sess.run(tf.reduce_sum(loss1)))
    print('total_loss2:',sess.run(tf.reduce_sum(loss2)))
    print('total_loss3:',sess.run(tf.reduce_sum(loss3)))


original= [[2, 0.5, 6], [0.1, 0, 3]]
scaled= [[ 0.01791432  0.00399722  0.97808844]
 [ 0.04980332  0.04506391  0.90513283]]
scaled2= [[ 0.21747023  0.21446465  0.56806517]
 [ 0.2300214   0.22893383  0.54104471]]
loss1= [ 0.02215516  3.09967351] 

loss2= [ 0.56551915  1.47432232] 

loss3= [ 0.02215518  3.09967351]
total_loss1: 3.12183
total_loss2: 2.03984
total_loss3: 3.12183



# 不使用标准one-hot，两个loss差距缩小。


In [12]:
#第一个label是下标2最大，下标0比较大，稍微接近logits中的分布,所以loss期望小。
#第二个label是下标1最大，下标0稍微大一点,下标2最小。而数据最大下标是2,差距悬殊，期望loss大。
labels = [[0.4,0.1,0.5],[0.3,0.6,0.1]]
logits = [[2,  0.5,6],
          [0.1,0,  3]]
loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
with tf.Session() as sess:
    print ("loss=",sess.run(loss),"\n")#实际上，两者loss差距比较小。
    print('total_loss:',sess.run(tf.reduce_sum(loss)))


loss= [ 2.17215538  2.76967359] 

total_loss: 4.94183


# sparse_softmax_cross_entropy_with_logits 
要求one-hot标签

In [11]:
#sparse
labels = [2,1] #其实是0 1 2 三个类。等价 第一行 001 第二行 010
logits = [[2,  0.5,6],
          [0.1,0,  3]]
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
with tf.Session() as sess:
    print ("loss=",sess.run(loss),"\n")#这个结果就比较符合预期了，loss1很小，loss2很大。
    print('total_loss:',sess.run(tf.reduce_sum(loss)))

loss= [ 0.02215516  3.09967351] 

total_loss: 3.12183
